In [1]:
import os
from glob import glob
from tqdm import tqdm
from pathlib import Path
import time

import pandas as pd
import numpy as np
from src.stats import calculate_p
print(os.getcwd())

/scratch4/lisik3/emcmaho7/SIfMRI_modeling/notebooks


In [2]:
model_class = 'VisionNeuralEncoding'
model_subpath = 'grouped_average'

In [3]:
top_dir = f'/home/emcmaho7/scratch4-lisik3/emcmaho7/SIfMRI_modeling/data/interim'

In [4]:
out_path = f'{top_dir}/ModelAveraging'
Path(out_path).mkdir(exist_ok=True, parents=True)

In [5]:
if model_subpath is not None: 
    file_path = f'{top_dir}/{model_class}/{model_subpath}/'
else: 
    file_path = f'{top_dir}/{model_class}'
cols2keep = ['voxel_id', 'layer_relative_depth',
             'train_score', 'test_score',
             'r_null_dist', 'r_var_dist']
voxel_id = 9539 #test voxel in EVC
files = glob(f'{file_path}/*.pkl.gz')
n_files = len(files)
print(f'{len(files)} files found')

342 files found


In [6]:
def sum_of_arrays(series):
    # Stack arrays vertically and compute sum along the first axis (rows)
    return np.nansum(np.vstack(series), axis=0)

def compute_confidence_intervals(arr):
    lower = np.nanpercentile(arr, 2.5)
    upper = np.nanpercentile(arr, 97.5)
    return lower, upper

def divide_array(arr):
    return arr / n_final_files

def calculate_p_df(row):
    r_value = row['test_score']  # The 'r' value for the current row
    r_null_array = row['r_null_dist']  # The 'r_null' array for the current row
    return calculate_p(r_null_array, r_value, n_perm_=len(r_null_array), H0_='greater')

In [7]:
df = None
n_final_files = 0
for file in tqdm(files, total=n_files, desc='Loading files'): 
    try: 
        pkl = pd.read_pickle(file)[cols2keep]

        # remove voxels not in roi
        if 'roi_name' in pkl.columns: 
            pkl = pkl.loc[pkl.roi_name != 'none'].reset_index()

        if df is None: 
            df = pkl
        else:
            #After the first file has been loaded, concatenate the data and add it together
            df = pd.concat([df, pkl])
            df = df.groupby('voxel_id').agg({
                                            'train_score': 'sum',
                                            'test_score': 'sum',
                                            'layer_relative_depth': 'sum',
                                            'r_null_dist': sum_of_arrays,
                                            'r_var_dist': sum_of_arrays
                                            }).reset_index()
        n_final_files += 1
    except:
        print(f'could not load {file}')
df.loc[df.voxel_id == voxel_id]

Loading files:  14%|█▎        | 47/342 [35:42<3:00:42, 36.75s/it]

could not load /home/emcmaho7/scratch4-lisik3/emcmaho7/SIfMRI_modeling/data/interim/VisionNeuralEncoding/grouped_average/model-timm_deit3_large_patch16_384.pkl.gz


Loading files:  26%|██▌       | 88/342 [1:06:21<2:22:09, 33.58s/it]

could not load /home/emcmaho7/scratch4-lisik3/emcmaho7/SIfMRI_modeling/data/interim/VisionNeuralEncoding/grouped_average/model-timm_ghostnet_100.pkl.gz


Loading files:  38%|███▊      | 129/342 [1:37:49<2:19:27, 39.28s/it]

...

could not load /home/emcmaho7/scratch4-lisik3/emcmaho7/SIfMRI_modeling/data/interim/VisionNeuralEncoding/grouped_average/model-timm_gernet_s.pkl.gz


Loading files:  90%|█████████ | 308/342 [3:52:39<19:45, 34.88s/it]

could not load /home/emcmaho7/scratch4-lisik3/emcmaho7/SIfMRI_modeling/data/interim/VisionNeuralEncoding/grouped_average/model-timm_gernet_m.pkl.gz


Loading files: 100%|██████████| 342/342 [4:17:27<00:00, 45.17s/it]


,voxel_id,train_score,test_score,layer_relative_depth,r_null_dist,r_var_dist
9539,9539,174.142181,141.718506,98.288762,"[24.27768939829548, -39.30439875662486, -26.34...","[78.70186874154024, 85.00554099213332, 106.027..."


In [8]:
# Get the mean by
columns_to_divide = ['train_score', 'test_score', 'layer_relative_depth', 'r_null_dist', 'r_var_dist']
for col in columns_to_divide:
    if isinstance(df[col][0], np.ndarray):
        # Apply division to arrays
        df[col] = df[col].apply(divide_array)
    else:
        # Apply division to numeric columns
        df[col] = df[col] / n_files
df.loc[df.voxel_id == voxel_id]

,voxel_id,train_score,test_score,layer_relative_depth,r_null_dist,r_var_dist
9539,9539,0.509188,0.414382,0.287394,"[0.0726876928092679, -0.1176778405886972, -0.0...","[0.23563433754952168, 0.2545076077608782, 0.31..."


In [9]:
df[['lower_ci', 'upper_ci']] = df['r_var_dist'].apply(lambda arr: pd.Series(compute_confidence_intervals(arr)))
df.loc[df.voxel_id == voxel_id]

,voxel_id,train_score,test_score,layer_relative_depth,r_null_dist,r_var_dist,lower_ci,upper_ci
9539,9539,0.509188,0.414382,0.287394,"[0.0726876928092679, -0.1176778405886972, -0.0...","[0.23563433754952168, 0.2545076077608782, 0.31...",0.093456,0.368982


In [10]:
df['p'] = df.apply(calculate_p_df, axis=1)
df.loc[df.voxel_id == voxel_id]

,voxel_id,train_score,test_score,layer_relative_depth,r_null_dist,r_var_dist,lower_ci,upper_ci,p
9539,9539,0.509188,0.414382,0.287394,"[0.0726876928092679, -0.1176778405886972, -0.0...","[0.23563433754952168, 0.2545076077608782, 0.31...",0.093456,0.368982,0.0


In [11]:
start_time = time.time()
df.to_pickle(f'{out_path}/{model_class}.pkl.gz')
end_time = time.time()
elapsed = end_time - start_time
elapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
print(f'Saved in {elapsed}!')

Saved in 00:01:47!
